In [ ]:
# 程序中使用到的自定义工具函数
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import joblib
import pandas as pd

def read_data(path: str):
    data = pd.read_excel(path, header=1) # 原始数据，包含表头
    s = data[0] # 硫含量
    RON = data[1] # 辛烷值
    # factors = data.drop(columns={0, 1, 2}) # 其它因素
    factors = factors.dropna(axis=1) # 去除空值列
    return s, RON, factors

# 获得选取的主要变量的数据
def use_data(data, column_set: list):
    return pd.DataFrame(data, columns=column_set)

# 训练神经网络模型
def train_mlp(X_train, y_train, num=300):
    # 数据标准化
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    # 定义神经网络
    clf = MLPRegressor(hidden_layer_sizes=(num,), random_state=0, max_iter=10000,
    shuffle=True)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_train)
    mse = mean_squared_error(y_train, y_pred)
    print(mse)
    print(clf.score(X_train, y_train))
    return clf, scaler
def train_rf(X_train, y_train):
    # 定义随机森林
    clf = RandomForestRegressor(n_estimators=500,
    random_state=0,
    max_features="auto",
    n_jobs=2)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_train)
    mse = mean_squared_error(y_train, y_pred)
    print(mse)
    print(clf.score(X_train, y_train))
    return clf
# 保存模型
def save_model(clf, name, scaler=None):
    joblib.dump(clf, "./model_1/" + name + ".pkl")
    if scaler:
        joblib.dump(scaler, "./model_1/" + name + ".s")
# 加载模型
def load_model(name, scaler=None):
    model = joblib.load("./model_1/" + name + ".pkl")
    if scaler:
        s = joblib.load("./model_1/" + name + ".s")
    return model

# 保存粒子群算法优化的样本操作函数
def save_pso_data(pso, i:int):
    joblib.dump(pso, "optimization_data/" + "sample_" + str(i) + ".con")

# 2 距离相关系数计算

In [2]:
ls

 驱动器 D 中的卷是 Data
 卷的序列号是 4248-79E8

 D:\cf-projects\01-科研\2021研究生数学建模\02-working\MathModel2021\Problem1 的目录

2021/10/14  19:36    <DIR>          .
2021/10/14  19:36    <DIR>          ..
2021/10/14  19:36    <DIR>          .ipynb_checkpoints
2021/10/14  19:29             6,176 2021-10-14指标提取.ipynb
2021/10/14  19:33             1,709 T1_corrs.py
               2 个文件          7,885 字节
               3 个目录 162,705,108,992 可用字节


In [3]:
pwd

'D:\\cf-projects\\01-科研\\2021研究生数学建模\\02-working\\MathModel2021\\Problem1'

In [6]:
# test
path = r'D:\cf-projects\01-科研\2021研究生数学建模\02-working\MathModel2021\Problem1\Molecular_Descriptor.xlsx'
raw_data = pd.read_excel(path) # 原始数据，包含表头
# data = raw_data.values # 去除表头的数据



<ipython-input-6-500435644613>:3: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  raw_data = pd.read_excel(path) # 原始数据，包含表头


In [5]:
# 变量之间的距离相关系数计算
# 相关性分析
from scipy.spatial.distance import pdist, squareform
import numpy as np
import pandas as pd

def distcorr(X, Y):
    X = np.atleast_1d(X)
    Y = np.atleast_1d(Y)
    if np.prod(X.shape) == len(X):
        X = X[:, None]
    if np.prod(Y.shape) == len(Y):
        Y = Y[:, None]
    X = np.atleast_2d(X)
    Y = np.atleast_2d(Y)
    n = X.shape[0]
    if Y.shape[0] != X.shape[0]:
        raise ValueError('Number of samples must match')
    a = squareform(pdist(X))
    b = squareform(pdist(Y))
    A = a - a.mean(axis=0)[None, :] - a.mean(axis=1)[:, None] + a.mean()
    B = b - b.mean(axis=0)[None, :] - b.mean(axis=1)[:, None] + b.mean()
    
    dcov2_xy = (A * B).sum() / float(n * n)
    dcov2_xx = (A * A).sum() / float(n * n)
    dcov2_yy = (B * B).sum() / float(n * n)
    dcor = np.sqrt(dcov2_xy) / np.sqrt(np.sqrt(dcov2_xx) * np.sqrt(dcov2_yy))
    
    return dcor

path = r'D:\cf-projects\01-科研\2021研究生数学建模\02-working\MathModel2021\Molecular_Descriptor.xlsx'
raw_data = pd.read_excel(path) # 原始数据，包含表头
data = raw_data.values # 去除表头的数据

# s = data[:, 0] # 硫含量
# RON = data[:, 1] # 辛烷值
factors = data[:, 2:] # 其它因素

m, n = factors.shape
corrs = np.zeros((n, n))
for i in range(n):
    print(i)
    for j in range(i, n):
        corrs[i, j] = distcorr(factors[:, i], factors[:, j])

print(corrs)
frame = pd.DataFrame(corrs)
frame.to_excel(r'D:\cf-projects\01-科研\2021研究生数学建模\02-working\MathModel2021\corr.xlsx', "Sheet2")

<ipython-input-5-1e8239ade644>:32: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  raw_data = pd.read_excel(path) # 原始数据，包含表头


0


<ipython-input-5-1e8239ade644>:27: RuntimeWarning: invalid value encountered in double_scalars
  dcor = np.sqrt(dcov2_xy) / np.sqrt(np.sqrt(dcov2_xx) * np.sqrt(dcov2_yy))


1


KeyboardInterrupt: 